In [1]:
pip install mlflow

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 7.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 KB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.4 MB/s eta 0:00:0000:01:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 703.4/703.4 KB 7.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 KB 11.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
# mlflow_register_demo.py
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Load dataset
X, y = load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = LinearRegression()
model.fit(X_train, y_train)

# MLflow experiment setup
mlflow.set_experiment("demo_model_registry")

with mlflow.start_run():
    # Log model params & metrics
    mlflow.log_param("model_type", "LinearRegression")
    mlflow.log_metric("score", model.score(X_test, y_test))

    # Log & Register model to MLflow Model Registry
    mlflow.sklearn.log_model(
        sk_model=model,
        name="DiabetesRegressionModel"
    )

print("✅ Model registered in MLflow Registry as 'DiabetesRegressionModel'")


2025/08/29 01:51:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Model registered in MLflow Registry as 'DiabetesRegressionModel'


In [8]:
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Load dataset
X, y = load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = LinearRegression()
model.fit(X_train, y_train)

# MLflow experiment setup
mlflow.set_experiment("demo_model_registry")

with mlflow.start_run() as run:
    # Log parameters and metrics
    mlflow.log_param("model_type", "LinearRegression")
    mlflow.log_metric("score", model.score(X_test, y_test))

    # Define artifact path (no slashes!)
    artifact_path = "diabetes_model"

    # Log the model
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path=artifact_path,
        input_example=X_test[:5],
        signature=infer_signature(X_test, model.predict(X_test))
    )

    # Register the model in the Model Registry
    result = mlflow.register_model(
        model_uri=f"runs:/{run.info.run_id}/{artifact_path}",
        name="DiabetesRegressionModel"  # ✅ Must NOT contain slashes
    )

print("✅ Model successfully registered in MLflow Registry as 'DiabetesRegressionModel'")


2025/08/29 01:53:09 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Registered model 'DiabetesRegressionModel' already exists. Creating a new version of this model...
2025/08/29 01:53:18 WARNING mlflow.tracking._model_registry.fluent: Run with id 74e5c98e3eb04b39930600378056d392 has no artifacts at artifact path 'diabetes_model', registering model based on models:/m-c7ccf75376ee4200a02c14cc8220056f instead


✅ Model successfully registered in MLflow Registry as 'DiabetesRegressionModel'


Created version '3' of model 'DiabetesRegressionModel'.


In [1]:
import time
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature
from mlflow.tracking import MlflowClient
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# === Step 1: Load and split dataset ===
X, y = load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# === Step 2: Train the model ===
model = LinearRegression()
model.fit(X_train, y_train)

# === Step 3: Setup MLflow experiment ===
mlflow.set_experiment("demo_model_registry")

with mlflow.start_run() as run:
    run_id = run.info.run_id

    # Log parameters and metrics
    mlflow.log_param("model_type", "LinearRegression")
    score = model.score(X_test, y_test)
    mlflow.log_metric("score", score)

    # Define artifact path (must be simple name, no slashes)
    artifact_path = "diabetes_model"

    # Log the model
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path=artifact_path,
        input_example=X_test[:5],
        signature=infer_signature(X_test, model.predict(X_test))
    )

    # Register the model
    result = mlflow.register_model(
        model_uri=f"runs:/{run_id}/{artifact_path}",
        name="DiabetesRegressionModel"
    )

# === Step 4: Wait briefly to avoid race condition ===
time.sleep(5)

# === Step 5: Set model stage and add descriptions ===
client = MlflowClient()

# Transition to "Staging"
client.transition_model_version_stage(
    name="DiabetesRegressionModel",
    version=result.version,
    stage="Staging",
    archive_existing_versions=True
)

# Add description to model version
client.update_model_version(
    name="DiabetesRegressionModel",
    version=result.version,
    description=f"Linear regression model trained on diabetes dataset. Score: {score:.4f}. Logged on 2025-08-29."
)

# Add description to registered model (across all versions)
client.update_registered_model(
    name="DiabetesRegressionModel",
    description="This model predicts diabetes progression using scikit-learn's LinearRegression."
)

print(f"✅ Model version {result.version} registered, transitioned to 'Staging', and described.")


2025/08/29 11:03:25 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Registered model 'DiabetesRegressionModel' already exists. Creating a new version of this model...
2025/08/29 11:03:38 WARNING mlflow.tracking._model_registry.fluent: Run with id 66cc627c13484ace9e4ce8422131aa7a has no artifacts at artifact path 'diabetes_model', registering model based on models:/m-5d4900061f10490e88689851aca1496d instead
Created version '5' of model 'DiabetesRegressionModel'.
/tmp/ipykernel_4628/2219106175.py:53: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


RepresenterError: ('cannot represent an object', <Metric: dataset_digest=None, dataset_name=None, key='score', model_id='m-5d4900061f10490e88689851aca1496d', run_id='66cc627c13484ace9e4ce8422131aa7a', step=0, timestamp=1756445605659, value=0.4526027629719195>)

In [2]:
import mlflow
import mlflow.sklearn
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from mlflow.models import infer_signature

def train_and_register_model():
    # Start an MLflow run
    with mlflow.start_run() as run:
        # Generate synthetic regression data
        X, y = make_regression(n_features=4, n_informative=2, random_state=0, shuffle=False)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Define and train the model
        params = {"max_depth": 2, "random_state": 42}
        model = RandomForestRegressor(**params)
        model.fit(X_train, y_train)

        # Evaluate the model
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)

        # Log parameters and metrics
        mlflow.log_params(params)
        mlflow.log_metrics({"mse": mse})

        # Infer the model signature (input/output schema)
        signature = infer_signature(X_train, model.predict(X_train))

        # Log and register the model
        mlflow.sklearn.log_model(
            sk_model=model,
            artifact_path="sklearn-model",
            signature=signature,
            registered_model_name="my-random-forest-reg-model"  # Name under which model will be registered
        )

        print(f"Model registered under name 'my-random-forest-reg-model', MSE: {mse}")

if __name__ == "__main__":
    train_and_register_model()


2025/08/29 11:20:59 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Model registered under name 'my-random-forest-reg-model', MSE: 356.6847104398046


Successfully registered model 'my-random-forest-reg-model'.
Created version '1' of model 'my-random-forest-reg-model'.


In [3]:
import mlflow
import mlflow.sklearn
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Start an MLflow run
with mlflow.start_run() as run:
    # Generate a simple classification dataset
    X, y = make_classification(n_samples=100, n_features=10, random_state=42)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Train a RandomForest model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Predict and evaluate the model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    # Log parameters and metrics
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("random_state", 42)
    mlflow.log_metric("accuracy", accuracy)

    # Log the model
    mlflow.sklearn.log_model(model, "random_forest_model")
    
    print(f"Model logged with accuracy: {accuracy:.4f}")


2025/08/29 12:03:00 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/29 12:03:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model logged with accuracy: 1.0000


In [4]:
import mlflow
import mlflow.sklearn
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from mlflow.models.signature import infer_signature

# Generate a simple classification dataset
X, y = make_classification(n_samples=100, n_features=10, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a RandomForest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict and evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

# Infer model signature (input/output schema)
signature = infer_signature(X_train, model.predict(X_train))

# Log parameters, metrics, and model with signature and input example
with mlflow.start_run() as run:
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("random_state", 42)
    mlflow.log_metric("accuracy", accuracy)

    # Log the model with signature and input example
    mlflow.sklearn.log_model(
        sk_model=model,
        name="random_forest_model",  # Use 'name' instead of deprecated 'artifact_path'
        input_example=X_train[:5],   # Provide an input example
        signature=signature          # Include the model signature
    )

    print(f"Model logged with accuracy: {accuracy:.4f}")


Model logged with accuracy: 1.0000


In [5]:
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Generate a simple classification dataset
X, y = make_classification(n_samples=100, n_features=10, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a RandomForest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict and evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

# Infer model signature (input/output schema)
signature = infer_signature(X_train, model.predict(X_train))

# Log parameters, metrics, and model with signature and input example
with mlflow.start_run() as run:
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("random_state", 42)
    mlflow.log_metric("accuracy", accuracy)

    # Log the model with signature and input example
    mlflow.sklearn.log_model(
        sk_model=model,
        name="random_forest_model",  # Use 'name' instead of deprecated 'artifact_path'
        input_example=X_train[:5],   # Provide an input example
        signature=signature          # Include the model signature
    )

    # Register the model in MLflow Model Registry
    model_uri = f"runs:/{run.info.run_id}/random_forest_model"
    mlflow.register_model(model_uri, "random_forest_model")

    print(f"Model logged with accuracy: {accuracy:.4f}")


Successfully registered model 'random_forest_model'.
2025/08/29 12:05:50 WARNING mlflow.tracking._model_registry.fluent: Run with id 353ee57f6b2242fcaf14284539e6e05a has no artifacts at artifact path 'random_forest_model', registering model based on models:/m-acdd8a0e9950453bb11a8589c64df8f6 instead


Model logged with accuracy: 1.0000


Created version '1' of model 'random_forest_model'.


In [7]:
import mlflow
mlflow.set_tracking_uri("http://localhost:5000")
